# COGS 108 - Data Checkpoint

# Names

- Victoria Thai
- Hannah Yick
- Jane Dinh
- Natasha Supangkat
- Gabriel Ramiro

<a id='research_question'></a>
# Research Question

What trends can be found between a county’s designation of a superfund site and its socioeconomic/demographic trends? Does the demographic makeup of a county influence the amount of time between Superfund designation and the completion of the remediation process?

# Dataset(s)

We will combine our datasets using the common variable of geographic location (county). By analyzing both aspects - Superfund activity and demographic breakdown - holistically, we will be able to better visualize and understand trends and potentially a relationship between the two.

1) **Superfund/National Priorities List (NPL) Sites**
- Number of observations: 1327 (as of April 26, 2021)
- Features: Region, state, site name, site ID, EPA ID, address, city, zip, county, federal facility indicator (whether or not the site is a federal site), latitude, longitude, listing date.
 - We will be focusing on the site name, county, and listing date.
- Summary: This dataset provides a comprehensive overview of the main characteristics of current Superfund sites on the National Priorities List. Geographical information (county) can help us better understand the spread of the data by location, and will allow us to combine with demographic data to explore trends. Additionally, we will use the listing date to determine the source of funding (based on the date of policy changes) as well as to measure how long the site has been active.
- Source: https://semspub.epa.gov/work/HQ/201371.pdf 

2) **2019 American Community Survey 5 Year Estimate by County**
- Number of observations: 3220 
- Features: Total population, race, per capita income median household income, poverty status by race. 
- Summary: This dataset provides demographic and socioeconomic data on each city’s population. This will help us understand population trends based on the NPL sites. 
- Source: https://www.socialexplorer.com/explore-maps



# Setup

In [49]:
#Imports 
import pandas as pd
import numpy as np

#Graphing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#Statistics
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

#Webscraping
import requests 
import bs4
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

# Read in the data and store it within a data frame
npl_df = npl_df = pd.read_csv('https://raw.githubusercontent.com/COGS108/group011_sp21/main/All%20current%20Final%20NPL%20Sites%20(FOIA%204).csv?token=AI2VKGXHTRP44IT6ZP5ASULASRZUM') 
gdrive_file_id = '1kNWY1Kc4RhQ9ttK7dTYtj0fhHE3BXOGh'
acs_df = pd.read_csv(f'https://docs.google.com/uc?id={gdrive_file_id}&export=download', encoding='ISO-8859-1')

# Data Cleaning

Describe your data cleaning steps here.

### National Priorities List (NPL) Superfund Data Cleaning

Let's take a look at the dataset we have for NPL sites:

In [50]:
npl_df

,Region,State,Site Name,Site ID,EPA ID,Address,City,Zip,County,FF Ind,Latitude,Longitude,NPL Status Date
0,1,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,100255,CTD980732333,ROUTE 44,BARKHAMSTED,6063,LITCHFIELD,N,41.893947,-72.989337,10/4/1989
1,1,CT,BEACON HEIGHTS LANDFILL,100180,CTD072122062,BLACKBERRY HILL ROAD,BEACON FALLS,6403,NEW HAVEN,N,41.431950,-73.035281,9/8/1983
2,1,CT,DURHAM MEADOWS,100108,CTD001452093,MAIN ST,DURHAM,6422,MIDDLESEX,N,41.481110,-72.681388,10/4/1989
3,1,CT,GALLUP'S QUARRY,100201,CTD108960972,ROUTE 12,PLAINFIELD,6374,WINDHAM,N,41.665281,-71.924161,10/4/1989
4,1,CT,KELLOGG-DEERING WELL FIELD,100252,CTD980670814,NORWALK WATER DEPARTMENT,NORWALK,6856,FAIRFIELD,N,41.130550,-73.431950,9/21/1984
5,1,CT,"LAUREL PARK, INC.",100232,CTD980521165,HUNTERS MTN RD,NAUGATUCK BOROUGH,6770,NEW HAVEN,N,41.476939,-73.071661,9/8/1983
6,1,CT,LINEMASTER SWITCH CORP.,100041,CTD001153923,29 PLAINE HILL ROAD,WOODSTOCK,6281,WINDHAM,N,41.940561,-71.966939,2/21/1990
7,1,CT,NEW LONDON SUBMARINE BASE,100261,CTD980906515,ROUTE 12 CRYSTAL LAKE RD,NEW LONDON,6349,NEW LONDON,Y,41.400139,-72.087081,8/30/1990
8,1,CT,PRECISION PLATING CORP.,100156,CTD051316313,1050 HARTFORD TURNPIKE ROAD,VERNON,6066,TOLLAND,N,41.849750,-72.447911,10/4/1989
9,1,CT,"RAYMARK INDUSTRIES, INC.",100094,CTD001186618,75 EAST MAIN STREET,STRATFORD,6614,FAIRFIELD,N,41.201278,-73.117222,4/25/1995


We removed the following columns from the dataset because they are irrelevant to our research.  

In [51]:
npl_df = npl_df.drop(["Region", "Site ID", "EPA ID", "Address", "Zip", "FF Ind", "Latitude", "Longitude"], axis=1)
npl_df

,State,Site Name,City,County,NPL Status Date
0,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,BARKHAMSTED,LITCHFIELD,10/4/1989
1,CT,BEACON HEIGHTS LANDFILL,BEACON FALLS,NEW HAVEN,9/8/1983
2,CT,DURHAM MEADOWS,DURHAM,MIDDLESEX,10/4/1989
3,CT,GALLUP'S QUARRY,PLAINFIELD,WINDHAM,10/4/1989
4,CT,KELLOGG-DEERING WELL FIELD,NORWALK,FAIRFIELD,9/21/1984
5,CT,"LAUREL PARK, INC.",NAUGATUCK BOROUGH,NEW HAVEN,9/8/1983
6,CT,LINEMASTER SWITCH CORP.,WOODSTOCK,WINDHAM,2/21/1990
7,CT,NEW LONDON SUBMARINE BASE,NEW LONDON,NEW LONDON,8/30/1990
8,CT,PRECISION PLATING CORP.,VERNON,TOLLAND,10/4/1989
9,CT,"RAYMARK INDUSTRIES, INC.",STRATFORD,FAIRFIELD,4/25/1995


***
### ACS Data Cleaning

Now let's take a look at our dataset for demographic data from the American Community Survey (ACS):

First, let's set the maximum rows displayed in the data frame to 5:

In [53]:
pd.options.display.max_rows = 5

In [54]:
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,Summary Level,Geographic Component,File identification,Logical Record Number,US,Census Region,Census Division,State (Census Code),State (FIPS Code),County of current residence,County Subdivision (FIPS),Place (FIPS Code),Place (State FIPS + Place FIPS),Census Tract,Block Group,Consolidated City,American Indian Area/Alaska Native Area/Hawaiian Home Land (Census),American Indian Area/Alaska Native Area/Hawaiian Home Land (FIPS),American Indian Trust Land/Hawaiian Home Land Indicator,American Indian Tribal Subdivision (Census),American Indian Tribal Subdivision (FIPS),Alaska Native Regional Corporation (FIPS),Metropolitan and Micropolitan Statistical Area,Combined Statistical Area,Metropolitan Division,Metropolitan Area Central City,Metropolitan/Micropolitan Indicator Flag,New England City and Town Area,New England City and Town Area.1,New England City and Town Area Division,Urban Area,Urban Area Central Place,Current Congressional District ***,State Legislative District Upper,State Legislative District Lower,Voting District,ZIP Code Tabulation Area (3-digit),5-digit ZIP Code Tabulation Area,Subminor Civil Division (FIPS),State-School District (Elementary),State-School District (Secondary),State-School District (Unified),Urban/Rural,Principal City Indicator,Traffic Analysis Zone,Urban Growth Area,Tribal Tract,Tribal Block Group,Public Use Microdata Area - 5% File,Public Use Microdata Area - 1% File,Total Population,Total Population.1,Population Density (Per Sq. Mile),Area (Land),Total Population:,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,Total Population: Two or More Races,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Under 

At a cursory look, the first row contains variable names we won't find useful. Hence we'll drop it then reset the index:

In [55]:
acs_df = acs_df.drop([0], axis=0)
acs_df.reset_index(drop=True, inplace=True)
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,Summary Level,Geographic Component,File identification,Logical Record Number,US,Census Region,Census Division,State (Census Code),State (FIPS Code),County of current residence,County Subdivision (FIPS),Place (FIPS Code),Place (State FIPS + Place FIPS),Census Tract,Block Group,Consolidated City,American Indian Area/Alaska Native Area/Hawaiian Home Land (Census),American Indian Area/Alaska Native Area/Hawaiian Home Land (FIPS),American Indian Trust Land/Hawaiian Home Land Indicator,American Indian Tribal Subdivision (Census),American Indian Tribal Subdivision (FIPS),Alaska Native Regional Corporation (FIPS),Metropolitan and Micropolitan Statistical Area,Combined Statistical Area,Metropolitan Division,Metropolitan Area Central City,Metropolitan/Micropolitan Indicator Flag,New England City and Town Area,New England City and Town Area.1,New England City and Town Area Division,Urban Area,Urban Area Central Place,Current Congressional District ***,State Legislative District Upper,State Legislative District Lower,Voting District,ZIP Code Tabulation Area (3-digit),5-digit ZIP Code Tabulation Area,Subminor Civil Division (FIPS),State-School District (Elementary),State-School District (Secondary),State-School District (Unified),Urban/Rural,Principal City Indicator,Traffic Analysis Zone,Urban Growth Area,Tribal Tract,Tribal Block Group,Public Use Microdata Area - 5% File,Public Use Microdata Area - 1% File,Total Population,Total Population.1,Population Density (Per Sq. Mile),Area (Land),Total Population:,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,Total Population: Two or More Races,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Under 

It also looks as if there are several columns/rows for which there are no data for, i.e. NaN, so we'll drop the columns that contain NaN values across all rows and vice versa: 

In [56]:
acs_df.dropna(axis = 0, how = 'all', inplace = True)
acs_df.dropna(axis = 1, how = 'all', inplace = True)

In [57]:
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,Summary Level,Geographic Component,File identification,Logical Record Number,State (FIPS Code),County of current residence,Total Population,Total Population.1,Population Density (Per Sq. Mile),Area (Land),Total Population:,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,Total Population: Two or More Races,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 18 to 64 for Whom Poverty Status Is Determined:,Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 65 and Over for Whom Poverty Status Is Determined:,Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,White Alone Population for Whom Poverty Status Is Determined:,White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,White Alone Population for Whom Poverty St

While no rows were dropped, for columns we went from 155 to 111. Let's take a look at the variables we have now: 

In [58]:
print(acs_df.columns.tolist())

['FIPS', 'Geographic Identifier', 'Area Name', 'Qualifying Name', 'State Postal Abbreviation', 'Summary Level', 'Geographic Component', 'File identification', 'Logical Record Number', 'State (FIPS Code)', 'County of current residence', 'Total Population', 'Total Population.1', 'Population Density (Per Sq. Mile)', 'Area (Land)', 'Total Population:', 'Total Population: White Alone', 'Total Population: Black or African American Alone', 'Total Population: American Indian and Alaska Native Alone', 'Total Population: Asian Alone', 'Total Population: Native Hawaiian and Other Pacific Islander Alone', 'Total Population: Some Other Race Alone', 'Total Population: Two or More Races', '% Total Population: White Alone', '% Total Population: Black or African American Alone', '% Total Population: American Indian and Alaska Native Alone', '% Total Population: Asian Alone', '% Total Population: Native Hawaiian and Other Pacific Islander Alone', '% Total Population: Some Other Race Alone', '% Total Pop

There seems to be several columns containing geographic information we won't find useful, so we'll drop those columns:

In [59]:
acs_df.drop(labels=['FIPS', 'Geographic Identifier', 'Summary Level', 'Geographic Component', 'File identification', 'Logical Record Number', 'State (FIPS Code)', 'County of current residence', 'Total Population.1', 'Population Density (Per Sq. Mile)', 'Area (Land)', 'Total Population:', 'Total Population: White Alone', 'Total Population: Black or African American Alone', 'Total Population: American Indian and Alaska Native Alone', 'Total Population: Asian Alone', 'Total Population: Native Hawaiian and Other Pacific Islander Alone', 'Total Population: Some Other Race Alone', 'Total Population: Two or More Races'], axis=1, inplace = True)

In [60]:
pd.options.display.max_columns = None
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 18 to 64 for Whom Poverty Status Is Determined:,Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 65 and Over for Whom Poverty Status Is Determined:,Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,White Alone Population for Whom Poverty Status Is Determined:,White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,White Alone Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,% White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% White Alone Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,Black or African American Alone Population for Whom Poverty Status Is Determined:,Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level,Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above

We're now down to 92 columns. Because our NPL dataset does not have for Puerto Rico, we're dropping the rows in this acs_data dataframe pertaining to Puerto Rico: 

In [61]:
acs_df.drop(index=acs_df[acs_df['State Postal Abbreviation'] == 'pr'].index, inplace=True)
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 18 to 64 for Whom Poverty Status Is Determined:,Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 18 to 64 for Whom Poverty Status Is Determined: At or Above Poverty Level,Population Age 65 and Over for Whom Poverty Status Is Determined:,Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,% Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 65 and Over for Whom Poverty Status Is Determined: At or Above Poverty Level,White Alone Population for Whom Poverty Status Is Determined:,White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,White Alone Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,% White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% White Alone Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,Black or African American Alone Population for Whom Poverty Status Is Determined:,Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level,Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above

We ended up dropping 78 rows.

Furthermore, because the scope of our research question only considers demographics living below the poverty level, we are dropping columns pertaining to demographics living at or above the poverty level:

In [62]:
acs_df.drop(list(acs_df.filter(regex = 'At or Above Poverty Level')), axis = 1, inplace = True)
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars),Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),Per Capita Income (In 2019 Inflation Adjusted Dollars).1,Per Capita Income (In 2019 Inflation Adjusted Dollars).2,Per Capita Income (In 2019 Inflation Adjusted Dollars).3,Per Capita Income (In 2019 Inflation Adjusted Dollars).4,Per Capita Income (In 2019 Inflation Adjusted Dollars).5,Per Capita Income (In 2019 Inflation Adjusted Dollars).6,Per Capita Income (In 2019 Inflation Adjusted Dollars).7,Per Capita Income (In 2019 Inflation Adjusted Dollars).8,Per Capita Income (In 2019 Inflation Adjusted Dollars).9,Population Under 18 Years of Age for Whom Poverty Status Is Determined:,Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,Population Age 18 to 64 for Whom Poverty Status Is Determined:,Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,Population Age 65 and Over for Whom Poverty Status Is Determined:,Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,White Alone Population for Whom Poverty Status Is Determined:,White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,Black or African American Alone Population for Whom Poverty Status Is Determined:,Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level,% Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level,American Indian and Alaska Native Alone Population for Whom Poverty Status Is Determined:,American Indian and Alaska Native Alone Population For&nbsp; Whom Poverty Status Is Determined: Income Below Poverty Level,% American Indian and Alaska Native Alone Population For&nbsp; Whom Poverty Status Is Determined: Income Below Poverty Level,Asian Alone Population for Whom Poverty Status Is Determined:,Asian Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Asian Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,Native Hawaiian and Other Pacific Islander Alone Population for Whom Poverty Status Is Determined:,Native Hawaiian and Other Pacific Islander Alone &nbsp; Population

We're now down to 68 columns. Let's take a closer look at the variable/column names alongside their indices: 

In [63]:
counter = 0
for col in acs_df.columns:
    print(counter, col)
    counter += 1

0 Area Name
1 Qualifying Name
2 State Postal Abbreviation
3 Total Population
4 % Total Population: White Alone
5 % Total Population: Black or African American Alone
6 % Total Population: American Indian and Alaska Native Alone
7 % Total Population: Asian Alone
8 % Total Population: Native Hawaiian and Other Pacific Islander Alone
9 % Total Population: Some Other Race Alone
10 % Total Population: Two or More Races
11 Median Household Income (In 2019 Inflation Adjusted Dollars)
12 Median Household Income (In 2019 Inflation Adjusted Dollars):
13 Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder
14 Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder
15 Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone  Householder
16 Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone
17 Median Household Income (In 2019 Inflation Adjusted Dollar

Our research question won't be considering per capita income on such a granular level, so we'll leave out the racial per capita income breakdown, i.e. columns 23-31. We also have a redundant column at index 11. So we will drop that as well. For columns 32-67, we have several redundant columns because raw numbers and percentages are included. We'll be only keeping the columns containing percentage data:

In [64]:
acs_df = acs_df.iloc[:, np.r_[0:11, 12:23, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67]]

In [65]:
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,% Total Population: Black or African American Alone,% Total Population: American Indian and Alaska Native Alone,% Total Population: Asian Alone,% Total Population: Native Hawaiian and Other Pacific Islander Alone,% Total Population: Some Other Race Alone,% Total Population: Two or More Races,Median Household Income (In 2019 Inflation Adjusted Dollars):,Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2019 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Per Capita Income (In 2019 Inflation Adjusted Dollars),% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 18 to 64 for Whom Poverty Status Is Determined: Living in Poverty,% Population Age 65 and Over for Whom Poverty Status Is Determined: Living in Poverty,% White Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Black or African American Alone Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level,% American Indian and Alaska Native Alone Population For&nbsp; Whom Poverty Status Is Determined: Income Below Poverty Level,% Asian Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Native Hawaiian and Other Pacific Islander Alone &nbsp; Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Some Other Race Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Two or More Races Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level,"% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level"
0,Autauga County,"Autauga County, Alabama",al,55380,76.79,19.03,0.25,1.03,0.05,0.68,2.16,58731,66540,28808,107656,2499,NaN,NaN,21189,86220,65992,29819,23.22,13.62,8.73,10.92,31.85,8.57,17.45,0,26.19,16.97,4.37,11.12
1,Baldwin County,"Baldwin County, Alabama",al,212830,86.21,9.26,0.77,0.93,0,1.14,1.69,58320,61505,36616,42179,47269,NaN,28837,61667,41851,61872,32626,13.36,10.24,7.39,8.58,24.67,20.36,7.87,0,26.28,10.79,18.41,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,Washakie County,"Washakie County, Wyoming",wy,8027,89.75,0.04,0.95,0,0,5.48,3.79,54158,53480,NaN,115526,NaN,NaN,62138,NaN,51071,54493,28101,16.43,10.78,2.93,10.27,NaN,16.67,NaN,NaN,21.91,0,8.9,11.24
3141,Weston County,"Weston County, Wyoming",wy,7049,97.42,0.23,0.17,0.77,0.14,0,1.28,57031,58081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58372,28531,18.69,12.64,15.18,13.91,100,0,0,0,NaN,60.24,15.38,13.91


We're now left with 34 variables/columns--more workable than the 155 we started off with.

Finally let's take a look at the breakdown of missing values from the resulting dataset:

In [66]:
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
acs_missingness_df = (acs_df.isna()
            .sum()
            .to_frame('missing_count')
            .assign(missing_ratio = lambda x: x['missing_count']/len(acs_df))
            .loc[acs_df.isna().any()] )
print(acs_missingness_df)

                                                                                                                                        missing_count  \
Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder                                                               2   
Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder                                        1218   
Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone  Householder                               1773   
Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone                                                                        1772   
Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone  Householder                      2894   
Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alon

There appears to be a lot of missingness for economic data when it comes to racial breakdowns, especially for more marginal groups such as Native Indian and Hawaiian groups. This may limit our analysis moving forward if we wanted to look beyond aggregate data and further analyze differences between racial groups. 
***